# 09 - Agentes con LangChain

## Curso de LLMs y Aplicaciones de IA

**Duración estimada:** 2.5-3 horas

---

## Índice

1. [Tipos de Agentes en LangChain](#tipos)
2. [Herramientas integradas](#herramientas)
3. [Agente con RAG (Retriever Tool)](#rag)
4. [Memoria en Agentes](#memoria)
5. [Salidas estructuradas](#estructuradas)
6. [Ejercicios prácticos](#ejercicios)

---

## Objetivos de aprendizaje

Al finalizar este notebook, serás capaz de:
- Usar diferentes tipos de agentes de LangChain
- Integrar herramientas de búsqueda y RAG
- Añadir memoria conversacional al agente
- Obtener salidas estructuradas con Pydantic

In [ ]:
# Install required libraries
!pip install -q langchain langchain-groq langchain-community langchain-huggingface
!pip install -q faiss-cpu sentence-transformers

In [ ]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

if 'GROQ_API_KEY' not in os.environ:
    os.environ['GROQ_API_KEY'] = getpass("Introduce tu GROQ API Key: ")

from langchain_groq import ChatGroq
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0)
print("LLM configurado ✓")

<a name="tipos"></a>
## 1. Tipos de Agentes en LangChain

| Tipo | Descripción | Uso |
|------|-------------|-----|
| **Tool Calling** | Usa function calling nativo | GPT-4, Llama 3 |
| **ReAct** | Thought-Action-Observation | General |
| **Structured Chat** | Para inputs estructurados | Múltiples parámetros |

<a name="herramientas"></a>
## 2. Herramientas Integradas

LangChain proporciona muchas herramientas pre-construidas.

In [ ]:
from langchain.tools import tool
from langchain_core.tools import Tool
from datetime import datetime
import random

# Custom tools
@tool
def calculator(expression: str) -> str:
    """Evalúa expresiones matemáticas. Ejemplo: '2+2' o '10*5'"""
    try:
        return str(eval(expression))
    except:
        return "Error en la expresión"

@tool
def get_datetime() -> str:
    """Obtiene fecha y hora actual."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def random_number(max_value: str) -> str:
    """Genera número aleatorio entre 1 y max_value."""
    try:
        return str(random.randint(1, int(max_value)))
    except:
        return "Error: proporciona un número"

@tool
def text_stats(text: str) -> str:
    """Devuelve estadísticas del texto: caracteres, palabras, oraciones."""
    chars = len(text)
    words = len(text.split())
    sentences = text.count('.') + text.count('!') + text.count('?')
    return f"Caracteres: {chars}, Palabras: {words}, Oraciones: {sentences}"

tools = [calculator, get_datetime, random_number, text_stats]
print(f"{len(tools)} herramientas configuradas")

In [ ]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

# Agent prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", """Eres un asistente con acceso a herramientas.
Usa las herramientas cuando sea necesario.
Siempre responde en español."""),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("Agente creado ✓")

In [ ]:
# Test agent
result = agent_executor.invoke({
    "input": "Genera un número aleatorio del 1 al 100 y multiplícalo por 7",
    "chat_history": []
})
print(f"\nRespuesta: {result['output']}")

<a name="rag"></a>
## 3. Agente con RAG (Retriever Tool)

Combinemos un agente con acceso a una base de conocimiento.

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.tools.retriever import create_retriever_tool

# Create knowledge base
company_docs = [
    Document(page_content="TechCorp ofrece tres planes: Básico (99€), Pro (299€), Enterprise."),
    Document(page_content="Horario de soporte: Lunes a Viernes, 9:00-18:00."),
    Document(page_content="Email soporte: soporte@techcorp.es. Tel: 900 123 456."),
    Document(page_content="Política devoluciones: 30 días para reembolso completo."),
    Document(page_content="TechCorp tiene oficinas en Madrid, Barcelona y Valencia."),
]

# Create vector store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(company_docs, embeddings)
retriever = vectorstore.as_retriever()

# Create retriever tool
retriever_tool = create_retriever_tool(
    retriever,
    "company_search",
    "Busca información sobre TechCorp: precios, horarios, contacto, políticas."
)

print("Retriever tool creado ✓")

In [ ]:
# Agent with RAG + other tools
rag_tools = [retriever_tool, calculator, get_datetime]

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """Eres un asistente de TechCorp.
Usa company_search para preguntas sobre la empresa.
Usa calculator para cálculos.
Responde siempre en español."""),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

rag_agent = create_tool_calling_agent(llm, rag_tools, rag_prompt)
rag_executor = AgentExecutor(agent=rag_agent, tools=rag_tools, verbose=True)

print("Agente RAG creado ✓")

In [ ]:
# Test RAG agent
result = rag_executor.invoke({
    "input": "¿Cuánto cuesta el plan Básico y cuánto pagaría por 6 meses?",
    "chat_history": []
})
print(f"\nRespuesta: {result['output']}")

<a name="memoria"></a>
## 4. Memoria en Agentes

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Session store
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Agent with memory
agent_with_memory = RunnableWithMessageHistory(
    rag_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

print("Agente con memoria configurado ✓")

In [ ]:
# Test memory
config = {"configurable": {"session_id": "user1"}}

# First question
r1 = agent_with_memory.invoke({"input": "Hola, me llamo Ana"}, config=config)
print(f"R1: {r1['output']}\n")

# Follow-up
r2 = agent_with_memory.invoke({"input": "¿Cuál es el horario de soporte?"}, config=config)
print(f"R2: {r2['output']}\n")

# Test memory
r3 = agent_with_memory.invoke({"input": "¿Cómo me llamo?"}, config=config)
print(f"R3: {r3['output']}")

<a name="estructuradas"></a>
## 5. Salidas Estructuradas

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

# Define output structure
class TaskAnalysis(BaseModel):
    task: str = Field(description="La tarea identificada")
    difficulty: str = Field(description="Dificultad: fácil, media, difícil")
    estimated_time: str = Field(description="Tiempo estimado")
    tools_needed: list = Field(description="Herramientas necesarias")

parser = PydanticOutputParser(pydantic_object=TaskAnalysis)

print("Parser configurado")
print(parser.get_format_instructions())

In [ ]:
# Use structured output
structured_prompt = f"""Analiza la siguiente tarea y responde en el formato especificado.

{parser.get_format_instructions()}

Tarea: Crear un dashboard de ventas que muestre gráficos interactivos y se actualice en tiempo real."""

response = llm.invoke(structured_prompt)
print(response.content)

<a name="ejercicios"></a>
## 6. Ejercicios Prácticos

### Ejercicio 1: Agente de soporte técnico

In [ ]:
# Exercise 1: Create a tech support agent
# Add documents about common tech problems
# Create appropriate tools
# Test with realistic support questions

## Resumen

En este notebook hemos aprendido:

1. **Tipos de agentes**: Tool calling, ReAct, Structured
2. **Herramientas**: Crear y usar herramientas personalizadas
3. **RAG + Agentes**: Combinar retrieval con capacidad de acción
4. **Memoria**: Mantener contexto entre turnos
5. **Salidas estructuradas**: Obtener datos parseables

En el siguiente notebook veremos **LangGraph** para crear flujos de trabajo más complejos.

---

## Referencias

- [LangChain Agents](https://python.langchain.com/docs/modules/agents/)
- [LangChain Tools](https://python.langchain.com/docs/modules/tools/)